In [66]:
class Crew:
    pass

class Rocket:
    pass

dir(Exception.args)

['__class__',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__objclass__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

# What is an Exception?

In [1]:
raise "Exception"

TypeError: exceptions must derive from BaseException

BaseException
 ├── BaseExceptionGroup
 ├── GeneratorExit
 ├── KeyboardInterrupt
 ├── SystemExit
 └── Exception
      ├── ArithmeticError
      │    ├── FloatingPointError
      │    ├── OverflowError
      │    └── ZeroDivisionError
      ├── AssertionError
      ├── AttributeError
      ├── EOFError
      ├── LookupError
      │    ├── IndexError
      │    └── KeyError

In [2]:
# Order of exceptions
def divide(a, b):
    return a/b

try:
    divide(1,0)
except Exception as e:
    print("Caught Exception")
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e


Caught Exception


In [3]:
formula = {"x": 1}

try:
    y = formula["y"]
except Exception as e:
    pass
print(y)

NameError: name 'y' is not defined

In [4]:
import sys
try:
    sys.exit()
except Exception as e:
    print("SystemExit caught")
except SystemExit as e:
    print("Raised SystemExit")

Raised SystemExit


In [44]:
import sys
try:
    sys.exit()
except Exception as e:
    print("SystemExit caught")
except BaseException as e:
    print("SystemExit caught by BaseException")    
except SystemExit as e:
    print("Raised SystemExit")

SystemExit caught by BaseException


In [5]:
inputs = [1, 0, "two"]

def divide(a, b):
    return a/b
for value in inputs:
    try:
        divide(1, value)
    except ZeroDivisionError as e:
        print("Can't divide by", value)
 

Can't divide by 0


TypeError: unsupported operand type(s) for /: 'int' and 'str'

In [9]:
inputs = [1, 0, "two"]

def divide(a, b):
    return a/b
for value in inputs:
    try:
        divide(1, value)
    except ZeroDivisionError as e:
        print("Can't divide by ", value)
    except TypeError as e:
        print("Can't divide by ", value)


Can't divide by  0
Can't divide by  two


In [54]:
inputs = [1, 0, "two"]

def divide(a, b):
    return a/b
for value in inputs:
    try:
        divide(1, value)
    except (ZeroDivisionError, TypeError) as e:
        print("Can't divide by", value)

Can't divide by 0
Can't divide by two


# Custom Exceptions

In [14]:
class FuelTypeException(Exception):
    def __init__(self, fuel_type):
        message = f"Invalid fuel type: {fuel_type}. Only 'Hydrogen' fuel is allowed."
        super().__init__(message)


In [15]:
class Rocket:
    
    
    def __init__(self, name, fuel, altitude):
        self.name = name
        self.fuel = fuel
        self.altitude = altitude
        self.allowed_fuel = "Hydrogen"
        
    def launch(self):
        if self.fuel > 0:
            print(f"{self.name} launched!")
            self.altitude += 100
            self.fuel -= 1
        else:
            print(f"{self.name} has no fuel left.")

    def refuel(self, amount, fuel_type):
        if fuel_type != self.allowed_fuel:
            raise FuelTypeException(fuel_type)
        self.fuel += amount
        print(f"{self.name} refueled. Fuel level is now {self.fuel}.")

    def status(self):
        print(f"{self.name} is at an altitude of {self.altitude} and has {self.fuel} units of fuel.")



In [23]:
rocket = Rocket("Falcon", 0, 0)
try:
    rocket.refuel(100, "Diesel")
except FuelTypeException as e:
    print(e)
    rocket.refuel(100, "Hydrogen")

Invalid fuel type: Diesel. Only 'Hydrogen' fuel is allowed.
Falcon refueled. Fuel level is now 100.


In [28]:
class FuelException(Exception):
    def __str__(self):
        return "Something is wrong in the fuel system"
        
class FuelTypeException(FuelException):
    def __init__(self, fuel_type):
        message = f"Invalid fuel type: {fuel_type}. Only 'Hydrogen' fuel is allowed."
        super().__init__(message)

In [29]:
rocket = Rocket("Falcon", 0, 0)
try:
    rocket.refuel(100, "Diesel")
except FuelException as e:
    print(e)
except FuelTypeException as e:
    print(e)
    rocket.refuel(100, "Hydrogen")

Something is wrong in the fuel system


## Explicit chained exceptions

In [ ]:
class RocketNotReadyError(Exception):
    pass


def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
        print("\tThe navigator's name is", crew[3])
    except IndexError as e:
        raise RocketNotReadyError('Crew is incomplete') from e

crew = ['John', 'Mary', 'Mike']
print('Final check procedure')

personnel_check()


In [ ]:
try:
    personnel_check()
except RocketNotReadyError as f:
    print('General exception: "{}", caused by "{}"'.format(f, f.__cause__))

# Supressing errors

In [32]:
# What if I don't care about an error? Can I just skip them?
inputs = [1, 0, "two", 3, 4]

def divide(a, b):
    return a/b

for value in inputs:
    try:
        print(divide(1, value))
    except (ZeroDivisionError, TypeError) as e:
        pass

1.0
0.3333333333333333
0.25


In [37]:
from contextlib import suppress

inputs = [1, 0, "two", 3, 4]

def divide(a, b):
    return a/b

for value in inputs:
    with suppress((ZeroDivisionError, TypeError)):
        print(divide(1, value))

1.0
0.3333333333333333
0.25


In [ ]:
# give example with Rocket code

# Low fuel error

# Restart engine function
# supress low fuel level error and try the restart anyway